Datset 

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder

# Load the mixed dataset
df_mixed_new = pd.read_csv('merged_shuffled_dataset.csv')

# Check the initial target audience distribution for verification
print("Initial target audience distribution:\n", df_mixed_new['target_audience'].value_counts())

# Correctly encode target labels using LabelEncoder
label_encoder = LabelEncoder()
df_mixed_new['target_label'] = label_encoder.fit_transform(df_mixed_new['target_audience'])

# Verify the label encoding
print("Label encoding classes:", label_encoder.classes_)  # This should output ['expert' 'layperson']
print("Encoded labels distribution:\n", df_mixed_new['target_label'].value_counts())

# Check the actual encodings to ensure correctness
encoded_expert = label_encoder.transform(['expert'])[0]
encoded_layperson = label_encoder.transform(['layperson'])[0]
print(f"Encoded 'expert' as: {encoded_expert}")  # Expected: 0
print(f"Encoded 'layperson' as: {encoded_layperson}")  # Expected: 1

# Define the custom dataset class
class ExplanationDataset(Dataset):
    def __init__(self, dataframe):
        self.labels = dataframe['target_label'].values
        self.texts = dataframe['explanation'].values
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], truncation=True, padding='max_length', max_length=512
        )
        item = {key: torch.tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Classification requires long tensor
        return item

# Function to train a model
def train_model(df, model_name):
    # Split the dataset into train and validation sets
    train_size = int(0.8 * len(df))
    val_size = len(df) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        ExplanationDataset(df), [train_size, val_size]
    )

    # Load BERT model for classification
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased', num_labels=len(label_encoder.classes_)
    )

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_{model_name}',
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs_{model_name}',
        logging_steps=10,
        evaluation_strategy="epoch",  
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,  
        metric_for_best_model="eval_loss", 
        logging_first_step=True
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Train the model
    trainer.train()

    # Save the model and tokenizer
    model.save_pretrained(f'bert-finetuned-{model_name}')
    tokenizer.save_pretrained(f'bert-finetuned-{model_name}')

# Train the model on the mixed dataset
train_model(df_mixed_new, 'Nego_BERT')


In [1]:
import os
import openai
import torch
from dotenv import load_dotenv
from transformers import BertTokenizer, BertForSequenceClassification, TextClassificationPipeline

load_dotenv() 
openai.api_key = os.getenv("OPENAI_API_KEY")



# Generating enriched explanation from mathematical sentence
domain = (
    "two agents representing two people living together while organizing a party negotiate over 6 issues: "
    "the food type, drinks type, location, type of invitations, music, and the clean-up service. Each issue "
    "further consists of 3 to 5 values, resulting in a domain with 3072 total possible outcomes."
)

def enrich_explanation(sentence):
    
    prompt = (
        f"Provide a clear and concise explanation of the following statement in just 1 or 2 lines. Consider the domain context:\n\n"
        f"{domain}\n\n"
        f"Statement: {sentence}\n\nExplanation:"
    )

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You're an expert assistant who provides clear and concise explanation"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400,
        temperature=0.7,
        top_p=0.9,
        frequency_penalty=0.0,
        presence_penalty=0.0,
    )

    enriched_explanation = response['choices'][0]['message']['content'].strip()
    return enriched_explanation


sentence = r"Ensures \(U_u(\omega_t^o) \) meets either a calculated statistical value or a specified minimum utility requirement in the initial interval \( [0.000, 0.0361) \)"
enriched_sentence = enrich_explanation(sentence)
print(f"Enriched Explanation:\n{enriched_sentence}\n")

/home/cim/pgt/mmac292/Dissertation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Enriched Explanation:
The statement means that the utility value \(U_u(\omega_t^o) \) should meet either a predetermined statistical value or a minimum utility threshold in the initial range of 0.000 to 0.0361.



In [2]:
def prompt_layperson(enriched_sentence):
    return (
        "Your task is to explain the following mathematical statement in very simple terms, suitable for someone without any technical background. The explanation should be clear, concise, and within 30 words. Avoid using any jargon or complex terms. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Examples of Clear Explanations for a Layperson:**\n"
        "1. The final price should match the average market price or include a discount, ensuring it is fair and competitive.\n"
        "2. In the first phase, the plan should improve basic features to be at least as good as a standard option.\n"
        "3. The service package should meet a basic quality level or reach a specific customer satisfaction score to ensure a good experience.\n"
        "4. The initial budget must be large enough to cover all estimated costs and any additional expenses.\n\n"
        "**Your Task:**\n"
        "Based on the mathematical statement provided, generate a clear and simple explanation suitable for a layperson, within 50 words."
    )


# Prompt for expert explanation


def prompt_expert(enriched_sentence):
    return (
        "Provide a detailed and technical explanation of the following mathematical statement for a domain expert. The explanation should be within 50 words. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Explanation for Domain Expert:**\n"
        "1. During the second interval [0.0361, 1.000], the utility of the opponent's offer \( U_u(\omega_t^o) \) must exceed the higher of a predefined threshold \( u \) or the quantile function \( U_{\Omega^o_t} \) at a specific time-dependent point.\n"
        "2. The initial evaluation phase requires the service package value \( V_s \) to surpass the minimum quality benchmark or meet a defined satisfaction threshold to ensure compliance with service standards.\n"
        "3. The order quantity \( Q_s \) must align with the highest value between the minimum stock level and a demand forecast quantile to optimize inventory management during the initial stocking phase.\n\n"
        "**Your Task:**\n"
        "Provide a similar style explanation suitable for an expert, within 50 words."
    )

In [3]:
def custom_explanation(sentence, target_audience, prompt_func, confidence_score=None, max_tokens=400, temperature=0.6, top_p=0.7, frequency_penalty=0.0, presence_penalty=0.0):
    # Generate the initial prompt based on the target audience
    prompt = prompt_func(enriched_sentence)

    # If confidence_score is provided, generate feedback
    if confidence_score is not None:
        feedback = generate_feedback(
            enriched_sentence, confidence_score, target_audience)
        prompt += f"\n\nFeedback for Improvement:\n{feedback}\n\nRefine the explanation based on the feedback."
    else:
        feedback = None

    # Use OpenAI's API to get the explanation
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert assistant. Your task is to provide clear and concise explanations for the specified audience."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    )

    # Extract the custom explanation from the response
    custom_explanation = response['choices'][0]['message']['content'].strip()

    # Check if the response is close to the token limit and add a note if it is
    if len(custom_explanation) >= max_tokens - 20:
        custom_explanation += " (response cut off, please refine or increase token limit)"

    return custom_explanation



In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the fine-tuned BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-finetuned-Nego_BERT')
tokenizer = BertTokenizer.from_pretrained('bert-finetuned-Nego_BERT')

# Function to get user's choice for the target audience
def get_user_choice():
    while True:
        choice = input("Choose the target audience (layperson/expert): ").strip().lower()
        if choice in ['layperson', 'expert']:
            return choice
        else:
            print("Invalid choice. Please enter 'layperson' or 'expert'.")

# Function to validate an explanation using the trained BERT model and return confidence score
def validate_explanation(explanation, intended_audience, max_length=512):
    try:
        # Ensure the model is in evaluation mode
        model.eval()

        # Tokenize the input explanation
        inputs = tokenizer(explanation, return_tensors="pt",
                           truncation=True, padding='max_length', max_length=max_length)

        # Predict the target audience
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get the logits and apply softmax to get probabilities
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probs, dim=1).item()
        confidence = probs.max().item()  # Get the confidence score of the prediction

        # Convert predicted label back to the target audience (0 for expert, 1 for layperson)
        predicted_audience = 'expert' if predicted_label == 0 else 'layperson'
        
        return predicted_audience, confidence
    except Exception as e:
        print(f"Error in validate_explanation: {e}")
        return None, 0.0  # Return zero confidence on error

# Function to generate feedback based on the intended and predicted audience
def generate_feedback(explanation, predicted_audience, target_audience):
    if predicted_audience == target_audience:
        return f"The explanation is suitable for a {target_audience}."
    else:
        return (f"The explanation is not suitable for a {target_audience}. "
                f"Please improve it to better match the needs of a {target_audience}.")


In [5]:
# Get user's choice for target audience
target_audience = get_user_choice()

# Loop until the explanation is suitable for the target audience
while True:
    # Generate a custom explanation (assuming you have this function defined)
    explanation = custom_explanation(
        enriched_sentence, target_audience, 
        prompt_layperson if target_audience == 'layperson' else prompt_expert, 
    )

    print(f"Generated Explanation for {target_audience}:\n{explanation}\n")

    # Validate the explanation with the model and get confidence score
    predicted_audience, confidence = validate_explanation(explanation, target_audience)

    if predicted_audience is None:
        print("Failed to validate the explanation. Please try again.")
        continue

    # Generate feedback based on the prediction
    feedback = generate_feedback(explanation, predicted_audience, target_audience)

    # Print confidence score
    print(f"Predicted Audience: {predicted_audience}, Confidence Score: {confidence:.2f}")

    if predicted_audience == target_audience:
        print(
            f"Final Explanation for {target_audience} based on enriched sentence:\n\n{explanation}\n"
            f"Confidence Score: {confidence:.2f}\n")
        break
    else:
        print(f"Explanation not suitable for {target_audience}. Feedback: {feedback}\nRefining explanation...\n")

Generated Explanation for layperson:
The value we're looking at should either match a set standard or be above a certain low point, within a specific small range from 0.000 to 0.0361.

Predicted Audience: layperson, Confidence Score: 0.98
Final Explanation for layperson based on enriched sentence:

The value we're looking at should either match a set standard or be above a certain low point, within a specific small range from 0.000 to 0.0361.
Confidence Score: 0.98

